In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
from sklearn.model_selection import KFold
import scipy as sc

import training_utils

In [2]:
img_size=(240, 320)
img_channels = 3
batch_size=32
epochs = 50
display_epochs = (0, 100)


In [3]:
model_name = 'Xception_like_global(32-64-126)2_(NaN).tf'

In [4]:
train_size = 1000
validation_size = 500
test_size = 500

# Load the DataSet

Dataset citation:
- From Semi-Supervised to Transfer Counting of Crowds
C. C. Loy, S. Gong, and T. Xiang
in Proceedings of IEEE International Conference on Computer Vision, pp. 2256-2263, 2013 (ICCV)
- Cumulative Attribute Space for Age and Crowd Density Estimation
K. Chen, S. Gong, T. Xiang, and C. C. Loy
in Proceedings of IEEE Conference on Computer Vision and Pattern Recognition, pp. 2467-2474, 2013 (CVPR, Oral)
- Crowd Counting and Profiling: Methodology and Evaluation
C. C. Loy, K. Chen, S. Gong, T. Xiang
in S. Ali, K. Nishino, D. Manocha, and M. Shah (Eds.), Modeling, Simulation and Visual Analysis of Crowds, Springer, vol. 11, pp. 347-382, 2013
- Feature Mining for Localised Crowd Counting
K. Chen, C. C. Loy, S. Gong, and T. Xiang
British Machine Vision Conference, 2012 (BMVC)

In [5]:
dataset_path = '/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/CrowdCounter'

In [6]:
labels_df = pd.read_csv(dataset_path + '/labels.csv')
labels_df['image_name'] = labels_df['id'].map('seq_{:06d}.jpg'.format)
labels_df.drop("id", axis=1,inplace=True)
display(labels_df)

,count,image_name
0,35,seq_000001.jpg
1,41,seq_000002.jpg
2,41,seq_000003.jpg
3,44,seq_000004.jpg
4,41,seq_000005.jpg
...,...,...
1995,27,seq_001996.jpg
1996,27,seq_001997.jpg
1997,25,seq_001998.jpg
1998,26,seq_001999.jpg


In [7]:
labels_df = labels_df.sample(frac=1).reset_index(drop=True)
display(labels_df)

,count,image_name
0,20,seq_000455.jpg
1,33,seq_001888.jpg
2,17,seq_000657.jpg
3,34,seq_000802.jpg
4,33,seq_001884.jpg
...,...,...
1995,29,seq_001421.jpg
1996,41,seq_000422.jpg
1997,42,seq_000710.jpg
1998,37,seq_001140.jpg


# Define the model

In [8]:
inputs = keras.Input(shape= img_size + (img_channels,))

## Custom Simple Covnet

Downsizing using strides instead of MaxPolling in order to conserve location data

In [9]:
def basic_covnet_block(input):
    x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(inputs)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
    return x

## Custom Depthwise Seperable Convolution

In [10]:
def custom_depthwise_conv_block(input, sizes, repeat_per_size, stride = 1, pooling = False, dropout = False, residual = False):
    x = input

    previous_block_activation = x
    
    for size in sizes:
        for i in range(repeat_per_size - 1):
            x = layers.BatchNormalization()(x)
            x  = layers.Activation("relu")(x)
            x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x  = layers.Activation("relu")(x)
        if dropout:
            x = layers.Dropout(0.5)(x)
        x = layers.SeparableConv2D(size, 3, strides=1 if pooling else stride, padding='same', use_bias=False)(x)
        if pooling:
            x = layers.MaxPooling2D(3, strides=stride, padding='same')(x)
        if residual:
            r = layers.SeparableConv2D(size, 1, strides=stride, padding="same")(
                previous_block_activation
            )
            x = layers.add([x, r])
            previous_block_activation = x



    return x
        

## Output Layer

In [11]:
def basic_dense_block(covnet_param, sizes, global_pooling = False):
    if global_pooling:
        x = layers.GlobalAveragePooling2D()(covnet_param)
    else:
        x = layers.Flatten()(covnet_param)
    x = layers.Dropout(0.5)(x)
    for size in sizes:
        x = layers.Dense(size, activation="relu")(x)
        x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1)(x)
    return outputs

In [12]:
def Xception_model():
    covnet = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(inputs)
    covnet = custom_depthwise_conv_block(covnet, [32, 64], 2, stride=2, pooling=True, residual=True)
    covnet = custom_depthwise_conv_block(covnet, [128], 2, residual=True)
    outputs = basic_dense_block(covnet, [], global_pooling=True)
    return keras.Model(inputs=inputs, outputs=outputs)

In [13]:
model = Xception_model()
model.summary()

2023-12-14 16:41:21.141774: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-14 16:41:21.141797: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-14 16:41:21.141807: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-14 16:41:21.141839: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-14 16:41:21.141855: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 240, 320, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 236, 316, 32)         2400      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 236, 316, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 236, 316, 32)         0         ['batch_normalization[0][0

# Train and compare different models

In [14]:
model.compile(loss="mse", optimizer="adam", metrics=["mae"])
# TODO: Try mae vs accuracy. mae should be better since we are adjusting it to get closer to the actual value

In [15]:
callbacks_list = [
#    keras.callbacks.EarlyStopping(
#        monitor="val_loss", patience=4
#    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor="val_loss",
        save_best_only=True
    ),
#    keras.callbacks.TensorBoard()
]

In [28]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)

result= next(kf.split(labels_df), None)
display(labels_df.iloc[result[0]])

test =  labels_df.iloc[result[1]]
display(test)
print(result[0])


,count,image_name
0,20,seq_000455.jpg
1,33,seq_001888.jpg
5,36,seq_000284.jpg
6,32,seq_000841.jpg
8,36,seq_001008.jpg
...,...,...
1992,28,seq_000406.jpg
1993,44,seq_000277.jpg
1994,29,seq_001938.jpg
1997,42,seq_000710.jpg


,count,image_name
2,17,seq_000657.jpg
3,34,seq_000802.jpg
4,33,seq_001884.jpg
7,31,seq_000382.jpg
37,25,seq_000886.jpg
...,...,...
1982,27,seq_001248.jpg
1983,30,seq_000116.jpg
1995,29,seq_001421.jpg
1996,41,seq_000422.jpg


[   0    1    5 ... 1994 1997 1999]


In [17]:
all(labels_df.iloc[result[0]]['image_name'].apply(lambda x: isinstance(x, str)))

True

In [37]:
def k_fold_validation(i_model):
    Wsave = i_model.get_weights()

    i_kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
    
    history_store = []
    
    for i_result in i_kf.split(labels_df):
    
        train = labels_df.iloc[i_result[0]]
        test =  labels_df.iloc[i_result[1]]
        
        print(i_result[0])
        print(i_result[1])
        
        display(labels_df)
        display(train)
        print(f'Train size: {len(train)}')
        display(test)
        print(f'Test size: {len(test)}')

        (train_generator, validation_generator) = training_utils.load_generators(train, test, dataset_path, batch_size, img_size)

        i_model.set_weights(Wsave)
        i_history = i_model.fit(train_generator,
                                epochs=epochs,
                                callbacks = callbacks_list,
                                validation_data=validation_generator,
                                )
        
        history_store.append(i_history)
        
    return history_store
    

In [38]:
history = k_fold_validation(model)

[   0    1    5 ... 1994 1997 1999]
[   2    3    4    7   37   45   52   61   65   66   68   69   76   80
   82   83   84   85   93  101  112  114  117  119  123  137  142  143
  145  154  155  157  161  173  176  180  181  197  199  215  226  230
  231  232  236  239  240  241  246  248  263  268  270  271  275  278
  280  284  288  294  305  316  323  332  344  352  354  355  356  361
  365  368  370  372  376  385  391  401  411  418  426  445  448  452
  455  456  458  459  465  468  470  472  479  481  484  488  492  494
  501  502  503  505  515  524  526  529  538  540  547  556  557  558
  560  570  577  580  581  582  586  598  600  606  610  611  615  619
  625  627  633  642  643  652  662  675  677  681  694  695  711  728
  735  738  748  768  770  776  786  802  821  823  825  826  835  837
  840  844  849  851  852  871  874  880  882  888  891  897  905  909
  916  923  928  945  951  952  961  969  978  980  981  985  991  996
 1006 1014 1023 1025 1040 1042 1046 1051 

,count,image_name
0,20,seq_000455.jpg
1,33,seq_001888.jpg
2,17,seq_000657.jpg
3,34,seq_000802.jpg
4,33,seq_001884.jpg
...,...,...
1995,29,seq_001421.jpg
1996,41,seq_000422.jpg
1997,42,seq_000710.jpg
1998,37,seq_001140.jpg


,count,image_name
0,20,seq_000455.jpg
1,33,seq_001888.jpg
5,36,seq_000284.jpg
6,32,seq_000841.jpg
8,36,seq_001008.jpg
...,...,...
1992,28,seq_000406.jpg
1993,44,seq_000277.jpg
1994,29,seq_001938.jpg
1997,42,seq_000710.jpg


Train size: 1600


,count,image_name
2,17,seq_000657.jpg
3,34,seq_000802.jpg
4,33,seq_001884.jpg
7,31,seq_000382.jpg
37,25,seq_000886.jpg
...,...,...
1982,27,seq_001248.jpg
1983,30,seq_000116.jpg
1995,29,seq_001421.jpg
1996,41,seq_000422.jpg


...

Test size: 400


,count,image_name
0,20,seq_000455.jpg
1,33,seq_001888.jpg
2,17,seq_000657.jpg
3,34,seq_000802.jpg
4,33,seq_001884.jpg
...,...,...
1995,29,seq_001421.jpg
1996,41,seq_000422.jpg
1997,42,seq_000710.jpg
1998,37,seq_001140.jpg


image_name
count
Found 1600 validated image filenames.
Found 400 validated image filenames.
Epoch 1/50
50/50 [==============================] - 29s 550ms/step - loss: 334.7704 - mae: 13.9538 - val_loss: 646.3046 - val_mae: 24.4455
Epoch 2/50
50/50 [==============================] - 27s 533ms/step - loss: 26.2691 - mae: 4.0843 - val_loss: 755.1987 - val_mae: 26.5893
Epoch 3/50
50/50 [==============================] - 28s 546ms/step - loss: 18.0416 - mae: 3.4145 - val_loss: 777.4079 - val_mae: 27.0063
Epoch 4/50
50/50 [==============================] - 27s 531ms/step - loss: 13.9858 - mae: 3.0158 - val_loss: 812.0717 - val_mae: 27.6377
Epoch 5/50
50/50 [==============================] - 27s 535ms/step - loss: 12.0514 - mae: 2.7819 - val_loss: 854.8868 - val_mae: 28.3989
Epoch 6/50
50/50 [==============================] - 26s 527ms/step - loss: 11.8133 - mae: 2.7487 - val_loss: 879.0723 - val_mae: 28.8201
Epoch 7/50
50/50 [==============================] - 26s 524ms/step - loss: 10.7491 -

INFO:tensorflow:Assets written to: Xception_like_global(32-64-126)2_(NaN).tf/assets


50/50 [==============================] - 29s 570ms/step - loss: 3.8409 - mae: 1.5230 - val_loss: 5.4764 - val_mae: 1.8690
Epoch 26/50
50/50 [==============================] - 27s 522ms/step - loss: 3.5725 - mae: 1.4948 - val_loss: 69.5857 - val_mae: 7.7964
Epoch 27/50
50/50 [==============================] - 26s 519ms/step - loss: 3.1202 - mae: 1.3855 - val_loss: 35.0056 - val_mae: 5.3352
Epoch 28/50
50/50 [==============================] - 26s 520ms/step - loss: 2.9904 - mae: 1.3934 - val_loss: 26.2755 - val_mae: 4.5670
Epoch 29/50
50/50 [==============================] - 26s 519ms/step - loss: 2.8474 - mae: 1.3524 - val_loss: 6.1488 - val_mae: 1.9579
Epoch 30/50
50/50 [==============================] - 26s 527ms/step - loss: 2.3250 - mae: 1.2196 - val_loss: 146.3159 - val_mae: 11.7747
Epoch 31/50
50/50 [==============================] - 27s 522ms/step - loss: 2.5760 - mae: 1.2667 - val_loss: 7.8841 - val_mae: 2.1918
Epoch 32/50
50/50 [==============================] - 27s 535ms/step 

In [ ]:
model.evaluate(validation_generator)

In [ ]:
model.evaluate(test_generator)

# Represent history

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['mae'][display_epochs[0]:display_epochs[1]])
plt.plot(history.history['val_mae'][display_epochs[0]:display_epochs[1]])
plt.title('model mean squared')
plt.ylabel('mean squared')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'][display_epochs[0]:display_epochs[1]])
plt.plot(history.history['val_loss'][display_epochs[0]:display_epochs[1]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Bibliografia

- Deep Learning with Python, Second Edition. François Chollet